# Men's NCAA Basketball Tournament Predictions

## Download data from Kaggle

I used the Kaggle API to download the data. This requires an account in order to obtain an API key and to accept the terms and conditions of the [Google Cloud & Men's 2019 NCAA Tournament ML Competition](https://www.kaggle.com/c/mens-machine-learning-competition-2019/).

In [150]:
import os
os.system('kaggle competitions download -c mens-machine-learning-competition-2019 --force')

SampleSubmissionStage1.csv: Skipping, found more recently modified local copy (use --force to force download)













100%|██████████| 13.7M/13.7M [00:03<00:00, 5.51MB/s]
100%|██████████| 19.9M/19.9M [00:03<00:00, 5.96MB/s]
100%|██████████| 20.0M/20.0M [00:04<00:00, 5.08MB/s]
100%|██████████| 19.9M/19.9M [00:04<00:00, 4.50MB/s]
100%|██████████| 20.3M/20.3M [00:03<00:00, 5.38MB/s]
100%|██████████| 21.0M/21.0M [00:04<00:00, 5.34MB/s]
100%|██████████| 20.4M/20.4M [00:04<00:00, 5.20MB/s]
100%|██████████| 21.2M/21.2M [00:03<00:00, 6.00MB/s]
100%|██████████| 20.5M/20.5M [00:04<00:00, 4.71MB/s]
100%|██████████| 20.6M/20.6M [00:04<00:00, 4.58MB/s]
100%|██████████| 4.25M/4.25M [00:00<00:00, 4.56MB/s]


We care mostly about the files in ```DataFiles.zip```, so we will only unzip this directory. All the others contain information about play-by-play events. It would be really cool to incorporate individual player stats into a ML algorithm, but for now, I will only use team stats for each individual game.

In [1]:
import zipfile
zip_ref = zipfile.ZipFile('DataFiles.zip', 'r')
zip_ref.extractall('DataFiles')
zip_ref.close()

zip_ref = zipfile.ZipFile('MasseyOrdinals.zip','r')
zip_ref.extractall('MasseyOrdinals')
zip_ref.close()

I like to use Pandas to work with tabular data. There are two types of game-by-game data for the NCAA: compact results which give the simple box scores for each game (teamIDs, scores, and who was the home team or if the game was played at a neutral site) and detailed results with box scores along with all the statisics like field goal attempts/completions, etc. The former goes back all the way to 1985 and the latter only to 2003. We can use all the box scores dating back from 1985 to establish a long enough baseline to determine accurage Elo scores.

In [2]:
import pandas as pd
import numpy as np
df = pd.concat([pd.read_csv('DataFiles/RegularSeasonCompactResults.csv'),
               pd.read_csv('DataFiles/NCAATourneyCompactResults.csv')]).reset_index(drop=True)
df = df.sort_values(by=['Season','DayNum'])

In [3]:
df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


Elo scores were first developed for chess but have been adopted to predict outcomes of other games and sports. I have chosen to use the NBA Elo scores developed by [FiveThirtyEight](https://fivethirtyeight.com/features/how-we-calculate-nba-elo-ratings/).

We can calculate the probability of winning based on the Elo score difference between any two teams.

Elo scores are updated after a game occurs based on how surprising the outcome is. The larger the difference in the scores, the larger the expected point spread is. If the actual point differential is larger than expected, the more the Elo scores change. Elo scores are a zero-sum game, so points are added to the team who beats the point spread while points are taken from the other team.

At the end of the season, the Elo scores revert back toward the mean of 1500.

Not all the teams have been Division 1 since 1985, so we will have to add in the new teams at the beginning of the season. For that, we will need the data file indicating their first D1 season. Every team's first D1 season (starting with 1985) will have an Elo score of 1300. The next block of code propagates the Elo scores in chronological order.

In [58]:
def forecast(df,K,HCA,R,return_prob=False):
    start = time.time()
    prob_all = np.zeros(len(df))
    WElo = np.zeros(len(df))
    LElo = np.zeros(len(df))
    
    team_elo = {t:1300 for t in df['WTeamID'].unique()}
    season = 1985
    for i,game in df.iterrows():
        
        if game['Season'] > season:
            for team,elo in team_elo.items():
                if team_elo[team] != 1300:
                    team_elo[team] = 1505*R + elo*(1-R)
            season = game['Season']
        
        locW = game['WLoc']
        if locW == 'H':
            value = HCA
        elif locW == 'A':
            value = -HCA
        else:
            value = 0
        
        elo_diff = team_elo[game['WTeamID']]-team_elo[game['LTeamID']]+value
        probW = 1 / (10**(-elo_diff/400) + 1)  
        
        MOV = game['WScore']=game['LScore']
    
        mult = (MOV+3)**0.8 / (7.5 + 0.006*elo_diff)
    
        shift = K*mult*(1-probW)
        
        WElo[i] = team_elo[game['WTeamID']]
        LElo[i] = team_elo[game['LTeamID']]
        
        team_elo[game['WTeamID']] = team_elo[game['WTeamID']] + shift
        team_elo[game['LTeamID']] = team_elo[game['LTeamID']] - shift
        
        prob_all[i] = probW

    score = (prob_all > 0.5).sum()/len(prob_all)
    sumprob = prob_all.sum()/len(prob_all)
    end = time.time()
    
    dt = end-start
        
    if return_prob:
        return prob_all, WElo, LElo
    else:
        result = {'K':K,'HCA':HCA,'R':R,'score':score,'sumprob':sumprob}
        print('K=%d,HCA=%d,R=1/%d...score=%0.3f,sumprob=%0.3f...Elapsed Time: %0.1f sec' % \
              (K,HCA,1/R,score,sumprob,dt))
        return result

The NBA Elo forecast from FiveThirtyEight used parameters K=20, HCA=100, R=1/3. The K factor determines how many elo points are transferred from the losing team to the winning team. HCA is the home court advantage, which gives extra Elo points to the home team. The R factor is how much the Elo scores revert back to the mean. For example, if a team had 1800 points at the end of the season, they would revert to 1700, subtract 1/3 of the difference from 1500.

We expect the NCAA to have similar values, but to be sure, we'll run the forecast with different parameters.

In [5]:
import time
import multiprocessing as mp

k = [10,20,30]
hca=[50,100,150]
r=[1/6,1/5,1/4,1/3]

start = time.time()
pool = mp.Pool(processes=4)
results = [pool.apply_async(forecast, args = (df,K,HCA,R)) \
           for K in k for HCA in hca for R in r]
output = [p.get() for p in results]

end = time.time()

t = (end-start)/60
print('Elapsed time: %0.1f min' % t)

K=10,HCA=50,R=1/6...score=0.724,sumprob=0.612...Elapsed Time: 26.2 sec
K=10,HCA=50,R=1/4...score=0.724,sumprob=0.606...Elapsed Time: 26.3 sec
K=10,HCA=50,R=1/3...score=0.724,sumprob=0.601...Elapsed Time: 26.4 sec
K=10,HCA=50,R=1/5...score=0.724,sumprob=0.609...Elapsed Time: 26.5 sec
K=10,HCA=100,R=1/6...score=0.730,sumprob=0.626...Elapsed Time: 27.4 sec
K=10,HCA=100,R=1/5...score=0.729,sumprob=0.623...Elapsed Time: 27.5 sec
K=10,HCA=100,R=1/3...score=0.727,sumprob=0.615...Elapsed Time: 27.4 sec
K=10,HCA=100,R=1/4...score=0.729,sumprob=0.620...Elapsed Time: 27.5 sec
K=10,HCA=150,R=1/6...score=0.722,sumprob=0.637...Elapsed Time: 26.0 sec
K=10,HCA=150,R=1/5...score=0.721,sumprob=0.635...Elapsed Time: 26.2 sec
K=10,HCA=150,R=1/4...score=0.720,sumprob=0.632...Elapsed Time: 26.2 sec
K=10,HCA=150,R=1/3...score=0.717,sumprob=0.628...Elapsed Time: 26.3 sec
K=20,HCA=50,R=1/6...score=0.713,sumprob=0.626...Elapsed Time: 24.9 sec
K=20,HCA=50,R=1/5...score=0.713,sumprob=0.624...Elapsed Time: 24.9 se

Process ForkPoolWorker-4:
Process ForkPoolWorker-3:
Process ForkPoolWorker-1:
Process ForkPoolWorker-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/anaconda3/lib/python3.6/m

In [6]:
elo_df = pd.DataFrame(output)

In [11]:
best = elo_df.loc[elo_df['score'].idxmax()]

In [59]:
df['Elo_Prob'], df['WElo'], df['LElo'] = forecast(df,best['K'],best['HCA'],best['R'],return_prob=True)

## Game statistics

In [64]:
data = pd.concat([pd.read_csv('DataFiles/RegularSeasonDetailedResults.csv'),
               pd.read_csv('DataFiles/NCAATourneyDetailedResults.csv')]).reset_index(drop=True)
data = data.sort_values(by=['Season','DayNum'])

Here are the statistics for each game since 2003.
- WFGM - field goals made (by the winning team)
- WFGA - field goals attempted (by the winning team)
- WFGM3 - three pointers made (by the winning team)
- WFGA3 - three pointers attempted (by the winning team)
- WFTM - free throws made (by the winning team)
- WFTA - free throws attempted (by the winning team)
- WOR - offensive rebounds (pulled by the winning team)
- WDR - defensive rebounds (pulled by the winning team)
- WAst - assists (by the winning team)
- WTO - turnovers committed (by the winning team)
- WStl - steals (accomplished by the winning team)
- WBlk - blocks (accomplished by the winning team)
- WPF - personal fouls committed (by the winning team)

And the same for the losing teams with an "L" in front.

Let's append the Elo scores and probabilities to this new table.

In [65]:
df_short = df[df['Season'] >= 2003].reset_index(drop=True)
data = pd.merge(df_short,data)

Right now, the Elo scores are a good measure of the historical performance of a basketball program; however, game stats will help us to determine how a team has been performing as of late. We'll do this by taking a rolling average of some of the statistics over previous games. We have to be careful to not include the current game stats in this average because it will help to reveal the outcome of the game, which we are trying to predict.

In [66]:
## Here are the statistics we want to create rolling means for
to_roll = ['FGM','FGA','FGM3','FGA3','FTM','FTA','OR','DR']

## We're going to create a dictionary of dictionaries that store the rolling means for each team.
##    The outer dictionary is for each individual team.
##    The inner dictionaries contain each statistic's rolling mean.
averages = dict()
for ID in data['WTeamID'].unique():
    averages[ID] = dict()
    team = data[(data['WTeamID'] == ID) | (data['LTeamID'] == ID)]
    for r in to_roll:
        ## Here we apply a function that selects the correct statistic based on whether the team is W or L.
        ## Method chaining: roll over last 4 games, take the mean, shift one down so that we don't include
        ##     the statistic for the game that is being predicted, backward fill for NaN
        averages[ID][r] = team.apply(lambda x: x['W%s' % r] if x['WTeamID'] == 1104 else x['L%s' % r],axis=1).\
            rolling(4).mean().shift(1).bfill()

The predictors we use for our comprehensive algorithm will be the difference of the winning team's statistic to that the losing team.

In [67]:
for r in to_roll:
    data['W%s_trend' % r] = data.apply(lambda row: averages[row['WTeamID']][r].loc[row.name],axis=1)
    data['L%s_trend' % r] = data.apply(lambda row: averages[row['LTeamID']][r].loc[row.name],axis=1)

In [68]:
for r in to_roll:
    data[r] = data['W%s_trend' % r] - data['L%s_trend' % r]

## Rankings

We have a compilation of rankings for each team given provided by Kenneth Massey going back to 2003. The effective date of the ranking is chosen so that predictions made based on those rankings can be made on that day.

I'll choose an approach based on how election polling reports go where we will use a rolling average in order to determine the effective rating of a team on a given day.

In order to do this in Pandas, we need to convert the ```Season,DayNum``` pairs into true dates. Luckily, we have also been given a file that gives day zero for every season.

In [35]:
rankings = pd.read_csv('MasseyOrdinals/MasseyOrdinals.csv')
rankings.head()

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
0,2003,35,SEL,1102,159
1,2003,35,SEL,1103,229
2,2003,35,SEL,1104,12
3,2003,35,SEL,1105,314
4,2003,35,SEL,1106,260


In [36]:
seasons = pd.read_csv('DataFiles/Seasons.csv',parse_dates=['DayZero'],index_col='Season')
seasons.head()

,DayZero,RegionW,RegionX,RegionY,RegionZ
Season,,,,,
1985,1984-10-29,East,West,Midwest,Southeast
1986,1985-10-28,East,Midwest,Southeast,West
1987,1986-10-27,East,Southeast,Midwest,West
1988,1987-11-02,East,Midwest,Southeast,West
1989,1988-10-31,East,West,Midwest,Southeast


Now, let's use both data frames to add a column to rankings that is the date of the released rankings.

In [37]:
rankings['Date'] = rankings.apply(lambda row: pd.to_timedelta(row['RankingDayNum'],unit='D') + \
                                  seasons.loc[row['Season'],'DayZero'],axis=1)    

Let's resample the rankings for each team by week and take a mean of the rankings. For the weeks when no rankings are available, we will interpolate the ranking value.

In [38]:
## save this in a dictionary with team ID as key
eff_rank = {t:rankings[rankings['TeamID'] == t].set_index('Date').resample('7D').mean().interpolate()['OrdinalRank'] \
               for t in rankings['TeamID'].unique()}

Now we need to match the ranks of the winning and losing teams in the main data frame.

In [69]:
data['Date'] = data.apply(lambda row: pd.to_timedelta(row['DayNum'],unit='D') \
                          + seasons.loc[row['Season'],'DayZero'],axis=1)

In [70]:
for i,game in data.iterrows():
    Wteam_rank = eff_rank[game['WTeamID']].reset_index()
    Widx = (Wteam_rank['Date'] - game['Date']).abs().idxmin()
    data.at[i,'Wrank'] = Wteam_rank.loc[Widx,'OrdinalRank']
    Lteam_rank = eff_rank[game['LTeamID']].reset_index()
    Lidx = (Lteam_rank['Date'] - game['Date']).abs().idxmin()
    data.at[i,'Lrank'] = Lteam_rank.loc[Lidx,'OrdinalRank']

In [71]:
data['RankDiff'] = data['Wrank'] - data['Lrank']

In [72]:
data.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,Elo_Prob,WElo,...,FGM3,FGA3,FTM,FTA,OR,DR,Date,Wrank,Lrank,RankDiff
0,2003,10,1104,68,1328,62,N,0,0.532471,1488.515970,...,1.0,4.75,7.25,9.00,5.00,3.75,2002-11-14,14.157895,22.157895,-8.000000
1,2003,10,1272,70,1393,63,N,0,0.545998,1579.975178,...,-0.5,-3.25,2.75,3.75,-2.75,0.25,2002-11-14,66.764706,76.823529,-10.058824
2,2003,11,1266,73,1437,61,N,0,0.312924,1438.790873,...,1.5,1.25,-6.25,-9.75,0.25,-1.25,2002-11-15,29.210526,42.411765,-13.201238
3,2003,11,1296,56,1457,50,N,0,0.944968,1683.740432,...,-1.5,-0.25,-4.00,-4.75,1.50,-2.75,2002-11-15,179.764706,231.176471,-51.411765
4,2003,11,1400,77,1208,71,N,0,0.826595,1476.718684,...,-2.0,-4.50,1.00,1.75,0.50,-2.50,2002-11-15,17.263158,63.764706,-46.501548


## Prepping for ML

The data came to us without a "target" value, as the first team listed is always the winning team, we have to create our own. One way would be to randomly split the data in two and for half the data swap the teams. If the winning team is in the first column, the target is 1, or else it is 0. Of course, in doing this, we also have to reverse the predictor values to reflect the change.

In [73]:
## shuffle the data by re-sampling all of it
data_shuffle = data.sample(frac=1,random_state=123)

## split the shuffled data in two
orig = data_shuffle.iloc[:len(data)//2,:]
rev = data_shuffle.iloc[len(data)//2:,:]

## reverse the values in the second half of the data
for r in to_roll:
    temp = -rev[r]
    rev = rev.drop(columns=r)
    rev[r] = temp
    
new_prob = 1-rev['Elo_Prob']
rev = rev.drop(columns=['Elo_Prob'])
rev['Elo_Prob'] = new_prob

new_rank = -rev['RankDiff']
rev = rev.drop(columns=['RankDiff'])
rev['RankDiff'] = new_rank

## order columns in rev
rev = rev[orig.columns]
orig = orig[orig.columns]

## Add columns for the target values
orig['result'] = 1
rev['result'] = 0

## Merge the two back together
data_target = pd.concat([orig,rev])
data_target = data_target.sort_values(by=['Season','DayNum'])

We will use all the values that we designed trending stats to along with the Elo probabilites and rank differentials. Then we split up the data frame of predictors ```X``` from the data frame of the target values ```y```.

In [74]:
predictors = to_roll + ['Elo_Prob','RankDiff']
X = data_target[predictors]
y = data_target['result']

In [45]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=123)

In [46]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(GradientBoostingClassifier(random_state=123),
                   param_grid = {'n_estimators':[100,200,300],
                                'min_samples_split':[2,3,4],
                                'max_depth':[3,4,5]})
grid.fit(X_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=123, subsample=1.0, verbose=0,
              warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [100, 200, 300], 'min_samples_split': [2, 3, 4], 'max_depth': [3, 4, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [47]:
clf = grid.best_estimator_

In [48]:
print('Train score: %0.2f' % clf.score(X_train,y_train))
print('Test score:  %0.2f' % clf.score(X_test,y_test))

Train score: 0.83
Test score:  0.82


In [49]:
%matplotlib inline
pd.DataFrame(clf.feature_importances_,index=X.columns).sort_values(by=0).plot(kind='barh',legend=False)

## Stage 1 submission: Historical tournaments

Kaggle requires us to predict a probability for the outcomes of every possible match up of teams for the last 4 NCAA tournaments (2014-2018). In order for us to do this, we need to grab the last row of stats for every team up until first day of the tournament. This must be submitted by March 16th, 2019, the day before Selection Sunday.

In [252]:
submit = []
for s in range(2014,2019):
    stats = []
    subset = data[(data['Season'] == s) & (data['DayNum'] <= 133)]
    for t in seeds[seeds['Season'] == s]['TeamID']:
        row = {'TeamID':t}
        team_stats = subset[(subset['WTeamID'] == t) | (subset['LTeamID'] == t)]
        team_stats = team_stats.loc[team_stats['Date'].idxmax()]
        if team_stats['WTeamID'] == t:
            for r in to_roll:
                row['%s_trend' % r] = team_stats['W%s_trend' % r]
            row['Elo'] = team_stats['WElo']
            row['Rank'] = team_stats['Wrank']
        else:
            for r in to_roll:
                row['%s_trend' % r] = team_stats['L%s_trend' % r]
            row['Elo'] = team_stats['LElo']
            row['Rank'] = team_stats['Lrank']
        stats.append(row)
    stats = pd.DataFrame(stats).sort_values('TeamID').set_index('TeamID')
    
    pairings = []
    for i in range(len(stats)):
        teamA = stats.iloc[i]
        for j in range(i+1,len(stats)):
            teamB = stats.iloc[j]
            x = teamA - teamB
            x['TeamA'] = teamA.name
            x['TeamB'] = teamB.name
            pairings.append(x)
    
    pairings = pd.DataFrame(pairings)
    
    pairings.columns = pairings.columns.str.replace('_trend','')
    pairings.columns = pairings.columns.str.replace('Rank','RankDiff')
    pairings.columns = pairings.columns.str.replace('Elo','EloDiff')
    pairings['Elo_Prob'] = pairings['EloDiff'].apply(lambda x: 1 / (10**(-x/400) + 1))
    
    Z = pairings[predictors]
    pairings['Pred'] = clf.predict_proba(Z)[:,1]
    
    
    pairings['ID'] = pairings.apply(lambda row: '%d_%d_%d' % (s,row['TeamA'],row['TeamB']),axis=1)
    
    submit.append(pairings[['ID','Pred']])
    print('Done with season %d' % s)

submit = pd.concat(submit).set_index('ID')

Done with season 2014
Done with season 2015
Done with season 2016
Done with season 2017
Done with season 2018


In [265]:
stamp = pd.Timestamp.now()
filename = 'submission_%d-%d-%d_%d:%d.csv' % (stamp.year,stamp.month,stamp.day,stamp.hour,stamp.minute)
submit.to_csv(filename)

message = 'First submission, Gradient Boosting Classifier with predictors ' + ','.join(predictors)

os.system('kaggle competitions submit -c mens-machine-learning-competition-2019 -f %s -m "%s"' % (filename,message))
print('Submission Complete!')

0